# Day 3: Geospatial web services and Interactive maps

### What is Geospatial Web Services? 

”A Web service that provides access to, or data processing on, geographic information. The OGC WebFeature Service (WFS), Web Map Service (WMS) are examples of geospatial Web service.” [IGI Global](https://www.igi-global.com/chapter/geospatial-interoperability/13798)

The Open Geospatial Consortium [OGC](https://www.ogc.org) is an international not for profit organization committed to making quality [open standards](https://www.ogc.org/docs/is) for the global geospatial community. 

Web Map Service (WMS)::: provides pre-assembled maps (may contain both vector and raster) to a requesting client. Map is usually generated upon request or cached.

Web Feature Service (WFS)::: allows clients to request raw vector data. May allow creating, editing, and deleting features on the server.


# Part 1: how to retrieve geographical data 

**We are going to retrieve geographical data from a popular platform called OpenStreetMap**

[OpenStreetMap](https://www.openstreetmap.org/) is a collaborative mapping project, which aims at creating a free editable map of the world. More about it [here](https://en.wikipedia.org/wiki/OpenStreetMap).


### Required modules for this session
- osm
- folium
- matplotlib
- ipyleaflet

OSMnx module allows you to retrieve OpenStreetMap data using [OverPass API](https://wiki.openstreetmap.org/wiki/Overpass_API).

We will download and visualize OSM data covering a specified area of interest in Copenhagen.

``OSMnx`` allows you to specify an address to retrieve the OpenStreetMap data around that area. Couldn’t be better!

In [ ]:
# import neccesary modules
# osmnx is easy to use for downloading OpenStreetMap data
import osmnx as ox
import matplotlib.pyplot as plt
%matplotlib inline

# Specify the name that is used to seach for the data
place_adr = "Christianshavn, Copenhagen, Denmark"

# Fetch OSM street network from the address
graph = ox.graph_from_address(place_adr, distance=1000)
type(graph)


In [ ]:
# Plot the streets
fig, ax = ox.plot_graph(graph)

In [ ]:
# Retrieve the footprint of the address
area = ox.gdf_from_place(place_adr)

# Retrieve buildings from the area
buildings = ox.footprints_from_address(place_adr, distance=1000)

ox.footprints_from_address(place_adr, distance=1000)


# What types are those?
print(type(area))
print(type(buildings))

In [ ]:
fig, ax = ox.plot_footprints(buildings)

Let’s also retrieve restaurants that are located on the area:

In [ ]:
# Retrieve restaurants
restaurants = ox.pois_from_address(place_adr, amenities=['restaurant'], distance= 1000)

# How many restaurants do we have?
len(restaurants)


Let’s explore what kind of attributes we have in our restaurants GeoDataFrame

In [ ]:
# Available columns
restaurants.columns

In [ ]:
buildings.columns

In [ ]:
# Select some useful cols and print
cols = ['alt_name', 'cuisine', 'opening_hours', 'outdoor_seating',
        'phone', 'wheelchair', 'geometry', 'payment:bitcoin']

restaurants[cols].head(12)


We can now plot all these different OSM layers by using the familiar ``plot()`` function of Geopandas. As you might remember, the street network data was not in GeoDataFrame format (it was networkx.MultiDiGraph). Luckily, osmnx provides a convenient function ``graph_to_gdfs()`` that can convert the graph into two separate GeoDataFrames where the first one contains the information about the nodes and the second one about the edge.



**Let’s extract the nodes and edges from the graph as GeoDataFrames:**


In [ ]:
# Retrieve nodes and edges
nodes, edges = ox.graph_to_gdfs(graph)
print("Nodes:\n", nodes.head(), '\n')
print("Edges:\n", edges.head(), '\n')
print("Type:", type(edges))

Let’s create a map out of the `streets`, `buildings`, `restaurants`, and the area of Polygon but let’s exclude the nodes (to keep the figure clearer).

In [ ]:
# Plot the footprint 

ax = area.plot(figsize=(60, 25),facecolor='white')

# Plot streets
edges.plot(ax=ax, linewidth=0.6, edgecolor=None)

# Plot buildings
buildings.plot(ax=ax, facecolor='gray', alpha=0.7)

# Plot restaurants
restaurant.plot(ax=ax, color='red', alpha=0.7, markersize=100)
plt.tight_layout()


# let's export a map from the area
outfp = "map.pdf"
plt.savefig(outfp, dpi=300)

In [ ]:
# Plot the footprint 

ax = area.plot(facecolor='white')

# Plot streets
edges.plot(ax=ax, linewidth=1, edgecolor=None)

# Plot buildings
buildings.plot(ax=ax, facecolor='black', alpha=0.7)

# Plot restaurants
restaurants.plot(ax=ax, color='red', alpha=0.7, markersize=10)
plt.tight_layout()

# let's export a map from the area
outfp = "map.pdf"
plt.savefig(outfp, dpi=300)

## 🏋 Exercise

find restaurants that they have ``payment:bitcoin`` and export a map of them.

# Part 2: how to make interactive maps

### Why interactive maps?
Static maps have been there forever, but it is the era of interactive maps so that the user can interact with them and explore more.

Most interactive maps are made with [Leaflet](https://leafletjs.com) or [OpenLayers](https://openlayers.org) JavaScipt library. JavaScript (JS) is a programming language mostly used for adding interactive content (zoom-able, pan-able maps) on webpages.

[Folium](https://python-visualization.github.io/folium/quickstart.html) library helps you to visualize data on an interactive Leaflet map.


In [ ]:
# First, we are going to make a simple interactive web-map with no data on it. 
# We are going to visualize OpenStreetMap for Copenhagen.

import folium

# Create a Map instance
m = folium.Map(location=[55.67, 12.57],
    zoom_start=14, control_scale=True)
m

### Let's check Folium's help 
You can see what tiles you can visualize as basemap in your map such as:
- "OpenStreetMap"
- "Mapbox Bright" (Limited levels of zoom for free tiles)
- "Mapbox Control Room" (Limited levels of zoom for free tiles)
- "Stamen" (Terrain, Toner, and Watercolor)
- "Cloudmade" (Must pass API key)
- "Mapbox" (Must pass API key)
- "CartoDB" (positron and dark_matter)

In [ ]:
help(folium.Map)

In [ ]:
import folium

# Create a Map instance and this time add more attributes
m = folium.Map(location=[55.67, 12.57], width='80%', height='80%', left='10%', top='0%', tiles="Stamen Toner", zoom_start=8, control_scale=True)
m

In [ ]:
# let’s export it to html file on your local driver

outfp = "base_map.html"
m.save(outfp)


Navigate to the file and open it with a text editor and check the script behind. You can also modify some settings there.

**Let's make one more map with AAU in the center**

In [ ]:
import folium

# Create a Map instance and this time add more attributes
m = folium.Map(location=[55.66, 12.57], width='100%', height='100%', left='0%', top='0%', tiles='Stamen Toner', zoom_start=15, control_scale=True, prefer_canvas=True)
m

In [ ]:
# Let’s add a simple marker to the webmap.

#Create a Map instance
m = folium.Map(location=[55.6505, 12.5429],
    zoom_start=16, control_scale=True)

# Add marker
# Run: help(folium.Icon) for more info about icons
folium.Marker(
    location=[55.6505, 12.5429],
    popup='Aalborg University',
    icon=folium.Icon(color='red', icon='info-sign'),
).add_to(m)

#Show map
m


In [ ]:
#you can read more about the markers here 
help(folium.Marker)

and some interesting demos [here](https://python-visualization.github.io/folium/quickstart.html)



Let's go to a mountainous area and enjoy a nice terrain map

In [ ]:
# Let's go to a mountainous area and enjoy a nice terrain map
m = folium.Map(location=[45.372, -121.6972], zoom_start=11, tiles='Stamen Terrain')

tooltip = 'Hit me!'

folium.Marker([45.3288, -121.6625], popup='<i>Mt. Hood Meadows</i>', tooltip=tooltip).add_to(m)
folium.Marker([45.3311, -121.7113], popup='<b>Timberline Lodge</b>', tooltip=tooltip).add_to(m)
folium.Marker([45.3231, -121.7143], popup='<i>Base camp</i>', tooltip=tooltip).add_to(m)
folium.Marker([45.3341, -121.7233], popup='<b>Jumping spot</b>', tooltip=tooltip).add_to(m)

m

In [ ]:

m = folium.Map(
    location=[55.6505, 12.5429],
    tiles='Stamen Toner',
    zoom_start=13
)

# the red one 
folium.Circle(
    radius=100,
    location=[55.6505, 12.5429],
    popup='AAU',
    color='crimson',
    fill=True,
).add_to(m)

# the blue one
folium.CircleMarker(
    location=[55.6505, 12.5429],
    radius=500,
    popup='Area sorrounding AAU',
    color='#3186cc',
    fill=False,
    fill_color='#3186cc'
).add_to(m)

# link to a color brewer for chosing color  http://colorbrewer2.org/# 

# a function to enable lat/lng pop-overs:
m.add_child(folium.LatLngPopup())

#x= m.add_child(folium.ClickForMarker(popup='Waypoint'))

m

In [ ]:
# Convert restaurants points to GeoJson
res_gjson = folium.features.GeoJson(restaurants, name = "Restaurants")


In [ ]:
# Create a Map instance
m = folium.Map(location=[55.6716, 12.5970], tiles = 'Stamen Toner', zoom_start=14, control_scale=True)

# Add points to the map instance
res_gjson.add_to(m)

# Alternative syntax for adding points to the map instance
#m.add_child(res_gjson)

#Show map
m

## Would you like to dig more into interactive maps?


[Ipyleaflet](https://github.com/jupyter-widgets/ipyleaflet) is also another great library for making interactive maps. Better try it out on your own. Some materials [here](https://github.com/jupyter-widgets/ipyleaflet)


Here are a few examples and interesting controls that you can add to your map

In [ ]:
# let's import some classes of ipyleaflet
from ipyleaflet import Map, basemaps, basemap_to_tiles

# we create a map instance just like we did with folium

m = Map(
    layers=(basemap_to_tiles(basemaps.NASAGIBS.ModisTerraTrueColorCR, "2020-05-28"), ),
    center=(53.204793, 12.121558),
    zoom = 6
)

m


### Note the basemap is a satellite image from [Modis-Terra](https://modis.gsfc.nasa.gov/about/)

In [ ]:
# in this exmaple, you can make a split map showing two different kinds of basemaps, left: ESRI world imagery, 
# Right:Modis-Terra

from ipyleaflet import Map, basemaps, basemap_to_tiles, SplitMapControl, FullScreenControl

#create map instance with center point and zoom level
m = Map(center=(42.6824, 365.581), zoom=5)

# set right and left windows' content, you can set the date of the image. Good luck to find a cloud free image
right_layer = basemap_to_tiles(basemaps.NASAGIBS.ModisTerraTrueColorCR, "2019-07-11")
left_layer = basemap_to_tiles(basemaps.Esri.WorldImagery)

# add split control
control = SplitMapControl(left_layer=left_layer, right_layer=right_layer)
m.add_control(control)

# add full screen control
m.add_control(FullScreenControl())

# show the map
m

``GeoData`` is an ``ipyleaflet`` class that allows you to visualize a GeoDataFrame on the Map.




In [ ]:
from ipyleaflet import Map, GeoData, basemaps, LayersControl, WidgetControl
from ipywidgets import IntSlider, jslink

import geopandas
import json

#let's read some coarse scale data from naturalearth, the data being rivers and countries boundaries
countries = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
rivers = geopandas.read_file("https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/physical/ne_10m_rivers_lake_centerlines.zip")

# add map instance
m = Map(center=(52.3,12.0), zoom = 3, basemap= basemaps.Esri.WorldTopoMap)

# add a control on the map to allow us to turn the two layers on and off...we also set the make up of the files e.g.,
# transparency, thickness, color, etc.

geo_data = GeoData(geo_dataframe = countries,
                   style={'color': 'black', 'fillColor': '#d7d7d7', 'opacity':0.2, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'Countries')

rivers_data = GeoData(geo_dataframe = rivers,
                   style={'color': 'red', 'opacity':3, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'Rivers')

# add them to the map instance
m.add_layer(rivers_data)
m.add_layer(geo_data)
m.add_control(LayersControl())

# let's add a zoom slider to the right conrner
zoom_slider = IntSlider(description='Zoom level:', min=0, max=15, value=6)
jslink((zoom_slider, 'value'), (m, 'zoom'))
widget_control1 = WidgetControl(widget=zoom_slider, position='topright')
m.add_control(widget_control1)

m

####  If you would like to visualize velocity related data e.g. wind, cars movement, trajectories, etc. you can learn from the following example

In [ ]:
from ipyleaflet import Map, TileLayer, basemaps
from ipyleaflet.velocity import Velocity
import xarray as xr
import os

# load a sample global wind data
if not os.path.exists('wind-global.nc'):
  url = 'https://github.com/benbovy/xvelmap/raw/master/notebooks/wind-global.nc'
  import requests
  r = requests.get(url)
  wind_data = r.content
  with open('wind-global.nc', 'wb') as f:
      f.write(wind_data)

# let's set the map instance
center = [56,13]
zoom = 7
m = Map(center=center, zoom=zoom, interpolation='nearest', basemap=basemaps.CartoDB.DarkMatter)

ds = xr.open_dataset('wind-global.nc')
display_options = {
    'velocityType': 'Global Wind',
    'displayPosition': 'bottomleft',
    'displayEmptyString': 'No wind data'
}
wind = Velocity(data=ds,
                zonal_speed='u_wind',
                meridional_speed='v_wind',
                latitude_dimension='lat',
                longitude_dimension='lon',
                velocity_scale=0.01,
                max_velocity=20,
                display_options=display_options)
m.add_layer(wind)

m

If you have a moving object or trajectory data, this might be of interest

In [ ]:
from ipyleaflet import Map, AntPath

m = Map(center=(51.332, 6.853), zoom=10)

ant_path = AntPath(
    locations=[
        [51.185, 6.773], [51.182, 6.752], [51.185, 6.733], [51.194, 6.729],
        [51.205, 6.732], [51.219, 6.723], [51.224, 6.723], [51.227, 6.728],
        [51.228, 6.734], [51.226, 6.742], [51.221, 6.752], [51.221, 6.758],
        [51.224, 6.765], [51.230, 6.768], [51.239, 6.765], [51.246, 6.758],
        [51.252, 6.745], [51.257, 6.724], [51.262, 6.711], [51.271, 6.701],
        [51.276, 6.702], [51.283, 6.710], [51.297, 6.725], [51.304, 6.732],
        [51.312, 6.735], [51.320, 6.734], [51.326, 6.726], [51.334, 6.713],
        [51.340, 6.696], [51.344, 6.678], [51.349, 6.662], [51.354, 6.655],
        [51.360, 6.655], [51.366, 6.662], [51.369, 6.675], [51.373, 6.704],
        [51.376, 6.715], [51.385, 6.732], [51.394, 6.741], [51.402, 6.743],
        [51.411, 6.742], [51.420, 6.733], [51.429, 6.718], [51.439, 6.711],
        [51.448, 6.716], [51.456, 6.724], [51.466, 6.719], [51.469, 6.713],
        [51.470, 6.701], [51.473, 6.686], [51.479, 6.680], [51.484, 6.680],
        [51.489, 6.685], [51.493, 6.700], [51.497, 6.714]
    ],
    dash_array=[1, 10],
    delay=1000,
    color='#7590ba',
    pulse_color='#3f6fba'
)

m.add_layer(ant_path)

m

OK. Here is one solution to the exercise in the beginning of this notebook.

In [ ]:
# Print only selected cols

grouped = restaurants.groupby('payment:bitcoin') 

grouped

In [ ]:
# Iterate over the group object 
for key, values in grouped: 
    bit_wel = values 
    
# Let's see what is the LAST item and key that we iterated 
print(len(values), ' restaurant(s) take(s) bitcoin.')
print(bit_wel)


In [ ]:
# Plot the footprint 

ax = area.plot(figsize=(60, 25),facecolor='white')

# Plot streets
edges.plot(ax=ax, linewidth=0.6, edgecolor=None)

# Plot buildings
buildings.plot(ax=ax, facecolor='gray', alpha=0.7)

# Plot restaurants
bit_wel.plot(ax=ax, color='red', alpha=0.7, markersize=100)
plt.tight_layout()


# let's export a map from the area
outfp = "map_bit.pdf"
plt.savefig(outfp, dpi=600)